In [1]:
import numpy as np
import pandas as pd 

# Data Loading

In [2]:
data = pd.read_csv('./data/training_new.csv', header=None, encoding='utf-8', names=["date", "office", "desti", "quantity"])
test = pd.read_csv('./data/submit_example.csv', encoding='utf-8', names=["date", "desti"])

In [3]:
data2 = pd.read_csv('./data/training_gi_new.csv', header=None, encoding='utf-8', names=["date", "desti", "quantity"])
test2 = pd.read_csv('./data/submit_example2.csv', encoding='utf-8', names=["date", "desti"])

In [4]:
data_pop = pd.read_csv('./data/seoul_population.txt', encoding='utf-8', sep='\t',usecols=['지역', '인구', '인구밀도(명/㎢)'])
data_pop = data_pop[1:].reset_index(drop=True)

In [5]:
data = data.groupby(['date', 'desti']).sum().reset_index()

## Date에 따른 컬럼 변환과 추가
'month', 'day', 'day_of_week', 'season'

In [6]:
data['date'] = pd.to_datetime(data["date"], format='%Y%m%d',)
test['date'] = pd.to_datetime(test["date"], format='%Y-%m-%d',)
data2['date'] = pd.to_datetime(data2["date"], format='%Y%m%d',)
test2['date'] = pd.to_datetime(test2["date"], format='%Y-%m-%d',)

In [7]:
# Train Data 요일 컬럼 추가
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['day_of_week'] = pd.to_datetime(data["date"], format='%Y%m%d',).dt.weekday_name

data2['month'] = data2['date'].dt.month
data2['day'] = data2['date'].dt.day
data2['day_of_week'] = pd.to_datetime(data2["date"], format='%Y%m%d',).dt.weekday_name

In [8]:
# Test Data 요일 컬럼 추가
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
test['day_of_week'] = pd.to_datetime(test["date"], format='%Y-%m-%d',).dt.weekday_name

test2['month'] = test2['date'].dt.month
test2['day'] = test2['date'].dt.day
test2['day_of_week'] = pd.to_datetime(test2["date"], format='%Y-%m-%d',).dt.weekday_name

In [9]:
def get_season(row):
    '''
    month에 따른 계절 추가
    '''
    if row['month'] >= 3 and row['month'] <= 5:
        return 'spring'
    elif row['month'] >= 6 and row['month'] <= 8:
        return 'summer'
    elif row['month'] >= 9 and row['month'] <= 11:
        return 'fall'
    else:
        return 'winter'

In [10]:
data['season'] = data.apply(get_season, axis=1)
test['season'] = test.apply(get_season, axis=1)
data2['season'] = data2.apply(get_season, axis=1)
test2['season'] = test2.apply(get_season, axis=1)

## 공휴일, 공휴일 다음날 컬럼 추가

In [11]:
def set_holiday(row):
    '''
    공휴일이면 1, 공휴일 아니면 0으로 설정
    '''
    if row['day_of_week'] == 'Saturday' or row['day_of_week'] == 'Sunday':
        return "holiday"
    elif (row['date'].month == 1 and row['date'].day == 1) or (row['date'].month == 2 and row['date'].day >= 15 and row['date'].day <= 18) or \
    (row['date'].month == 3 and row['date'].day == 1) or (row['date'].month == 5 and row['date'].day >= 5 and row['date'].day <= 7) or \
    (row['date'].month == 5 and row['date'].day == 22) or (row['date'].month == 6 and row['date'].day == 6) or (row['date'].month == 6 and row['date'].day == 13) or \
    (row['date'].month == 8 and row['date'].day == 15) or (row['date'].month == 9 and row['date'].day >= 22 and row['date'].day <= 26) or \
    (row['date'].month == 10 and row['date'].day == 3) or (row['date'].month == 10 and row['date'].day == 9) or (row['date'].month == 12 and row['date'].day == 25):
        return "holiday"
    else:
        return "noholiday"

In [12]:
data['holiday'] = data.apply(set_holiday, axis=1)
test['holiday'] = test.apply(set_holiday, axis=1)
data2['holiday'] = data2.apply(set_holiday, axis=1)
test2['holiday'] = test2.apply(set_holiday, axis=1)

In [13]:
def set_nextday(row):
    '''
    공휴일 다음날이면 1, 공휴일 다음날이 아니면 0으로 설정
    '''
    if row['day_of_week'] == 'Saturday' or row['day_of_week'] == 'Sunday':
        return "nonextday"
    elif (row['date'].month == 1 and row['date'].day == 1) or (row['date'].month == 2 and row['date'].day >= 15 and row['date'].day <= 18) or \
    (row['date'].month == 3 and row['date'].day == 1) or (row['date'].month == 5 and row['date'].day >= 5 and row['date'].day <= 7) or \
    (row['date'].month == 5 and row['date'].day == 22) or (row['date'].month == 6 and row['date'].day == 6) or (row['date'].month == 6 and row['date'].day == 13) or \
    (row['date'].month == 8 and row['date'].day == 15) or (row['date'].month == 9 and row['date'].day >= 22 and row['date'].day <= 26) or \
    (row['date'].month == 10 and row['date'].day == 3) or (row['date'].month == 10 and row['date'].day == 9) or (row['date'].month == 12 and row['date'].day == 25):
        return "nonextday"
    elif row['day_of_week'] == 'Monday':
        return 'nextday'
    elif (row['date'].month == 1 and row['date'].day == 2) or (row['date'].month == 2 and row['date'].day >= 5 and row['date'].day <= 9) or \
    (row['date'].month == 2 and row['date'].day == 19) or (row['date'].month == 5 and row['date'].day == 8) or \
    (row['date'].month == 5 and row['date'].day == 23) or (row['date'].month == 6 and row['date'].day == 7) or (row['date'].month == 6 and row['date'].day == 14) or \
    (row['date'].month == 8 and row['date'].day == 16) or (row['date'].month == 9 and row['date'].day == 27) or (row['date'].month == 9 and row['date'].day >= 10 and row['date'].day <= 14) or \
    (row['date'].month == 10 and row['date'].day == 4) or (row['date'].month == 10 and row['date'].day == 10) or (row['date'].month == 12 and row['date'].day == 26):
        return 'nextday'
    else:
        return 'nonextday'

In [14]:
data['nextday'] = data.apply(set_nextday, axis=1)
test['nextday'] = test.apply(set_nextday, axis=1)
data2['nextday'] = data2.apply(set_nextday, axis=1)
test2['nextday'] = test2.apply(set_nextday, axis=1)

In [15]:
# data_pop_join = data.join(data_pop.set_index('desti'), on='desti')

# data_pop_join['population'] = data_pop_join.population.astype(int)
# data_pop_join

# def set_ratio(row):
#     return row['quantity'] / row['population']

# data_pop_join['ratio'] = data_pop_join.apply(set_ratio, axis=1)
# data_pop_join


## Train Data 생성

In [16]:
data.head()

,date,desti,quantity,month,day,day_of_week,season,holiday,nextday
0,2017-12-01,0,5829,12,1,Friday,winter,noholiday,nonextday
1,2017-12-01,1,7455,12,1,Friday,winter,noholiday,nonextday
2,2017-12-01,2,10086,12,1,Friday,winter,noholiday,nonextday
3,2017-12-01,3,5929,12,1,Friday,winter,noholiday,nonextday
4,2017-12-01,4,14913,12,1,Friday,winter,noholiday,nonextday


In [17]:
temp_data = data.drop(["quantity", "date"], axis=1).copy()

In [18]:
train_data = data.drop(["quantity", "date"], axis=1) # 훈련 세트를 위해 레이블 삭제
train_data_labels = data["quantity"].copy()
train_data_cat = train_data[['day_of_week', 'season', 'holiday', 'nextday']]
train_data_num = train_data.drop(columns=['day_of_week', 'season', 'holiday', 'nextday'])

In [19]:
test_data = test.drop(["date"], axis=1)
test_data_cat = test_data[['day_of_week', 'season', 'holiday', 'nextday']]
test_data_num = test_data.drop(columns=['day_of_week', 'season', 'holiday', 'nextday'])

In [20]:
train_data_2 = data2.drop(["quantity", "date"], axis=1) # 훈련 세트를 위해 레이블 삭제
train_data_labels_2 = data2["quantity"].copy()

train_data_cat_2 = train_data_2[['day_of_week', 'season', 'holiday', 'nextday']]
train_data_num_2 = train_data_2.drop(columns=['day_of_week', 'season', 'holiday', 'nextday'])

In [21]:
test_data_2 = test2.drop(["date"], axis=1)
test_data_cat_2 = test_data_2[['day_of_week', 'season', 'holiday', 'nextday']]
test_data_num_2 = test_data_2.drop(columns=['day_of_week', 'season', 'holiday', 'nextday'])

In [22]:
test_data.head()

,desti,month,day,day_of_week,season,holiday,nextday
0,21,12,24,Sunday,winter,holiday,nonextday
1,16,12,24,Sunday,winter,holiday,nonextday
2,20,12,24,Sunday,winter,holiday,nonextday
3,8,12,24,Sunday,winter,holiday,nonextday
4,19,12,24,Sunday,winter,holiday,nonextday


In [23]:
test_data_2.head()

,desti,month,day,day_of_week,season,holiday,nextday
0,0,9,11,Tuesday,fall,noholiday,nextday
1,1,9,11,Tuesday,fall,noholiday,nextday
2,2,9,11,Tuesday,fall,noholiday,nextday
3,3,9,11,Tuesday,fall,noholiday,nextday
4,4,9,11,Tuesday,fall,noholiday,nextday


## Encoding과 Scaling

In [45]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion
from sklearn.cluster import KMeans



num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('std_scaler', StandardScaler()),
        ('poly', PolynomialFeatures(2))
#         ('minmax_scaler', MinMaxScaler()),
    ])


num_attribs = list(train_data_num)
cat_attribs = ['day_of_week', 'season', 'holiday', 'nextday']

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(sparse=False), cat_attribs),
    ])


In [46]:
train_data_prepared = full_pipeline.fit_transform(train_data)
train_data_prepared_2 = full_pipeline.fit_transform(train_data_2)

test_data_prepared = full_pipeline.transform(test_data)
test_data_prepared_2 = full_pipeline.transform(test_data_2)

In [47]:
train_data_prepared[0]

array([ 1.        , -1.66387623,  1.63216768, -1.6580551 ,  2.76848412,
       -2.71572501,  2.75879848,  2.66397134, -2.70622395,  2.74914672,
        1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  1.        ,  0.        ,  1.        ])

## 선형 회귀 모델

In [449]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(train_data_prepared, train_data_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [450]:
# 훈련 샘플 몇 개를 사용해 전체 파이프라인을 적용
some_data = temp_data.iloc[:5]
some_labels = train_data_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("예측:", lin_reg.predict(some_data_prepared))
print(some_labels)

예측: [7179.00886244 7220.37462591 7261.74038938 7303.10615285 7344.47191632]
0     5829
1     7455
2    10086
3     5929
4    14913
Name: quantity, dtype: int64


## 랜덤포레스트 모델

In [62]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

forest_reg = RandomForestRegressor(n_estimators=150, bootstrap=True)

In [63]:
forest_reg.fit(train_data_prepared_2, train_data_labels_2)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [64]:
result = forest_reg.predict(test_data_prepared_2)

In [ ]:
forest_reg.fit(train_data_prepared, train_data_labels)

In [ ]:
result = forest_reg.predict(test_data_prepared)

In [453]:
# 훈련 샘플 몇 개를 사용해 테스트하기
some_data = temp_data.iloc[:5]
some_labels = train_data_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("예측:", forest_reg.predict(some_data_prepared))
print(some_labels)

예측: [ 5910.1  7595.  10086.   6039.1 15518.7]
0     5829
1     7455
2    10086
3     5929
4    14913
Name: quantity, dtype: int64


In [28]:
# 랜덤포레스트 최적화 하이퍼 파라미터 서치
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10, 20]
min_samples_leaf = [1, 2, 4, 10, 50]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [30]:
rf_random = RandomizedSearchCV(estimator = forest_reg, param_distributions = random_grid, n_iter = 200, cv = 5, verbose=2, random_state=42, n_jobs = -1, scoring='neg_mean_squared_error')

rf_random.fit(train_data_prepared, train_data_labels)

Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 26.4min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 62.3min
[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed: 104.7min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False),
          fit_params=None, iid='warn', n_iter=200, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10, 20], 'min_samples_leaf': [1, 2, 4, 10, 50], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring='neg_mean_squared_error',
          verbose=2)

In [ ]:
rf_random = RandomizedSearchCV(estimator = forest_reg, param_distributions = random_grid, n_iter = 200, cv = 5, verbose=2, random_state=42, n_jobs = -1, scoring='neg_mean_squared_error')

rf_random.fit(train_data_prepared_2, train_data_labels_2)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
result = rf_random.best_estimator_.predict(test_data_prepared_2)

In [ ]:
result = np.around(result)

In [ ]:
result

## K-means, Random Forest 통합 모델

In [ ]:
union  = Pipeline([
    ('k_means_5', KMeans(n_clusters=5)),
    ('random_forest', RandomizedSearchCV(estimator = forest_reg, param_distributions = random_grid, n_iter = 200, cv = 3, verbose=2, random_state=42, n_jobs = -1))
])

In [ ]:
union.fit(train_data_prepared, train_data_labels)

In [464]:
print("예측:", forest_reg.predict(test_data_prepared))
result = forest_reg.predict(test_data_prepared)

예측: [  597.7   336.9   338.9   145.4   355.4   188.3   289.    357.1   337.9
   337.4   246.1   201.7   172.    271.    177.6   295.6   238.6   358.6
   211.6   771.6   401.8   313.3  1589.9   398.1   621.2   359.4   182.6
   812.8   307.4   247.5   315.3   290.8   234.5   344.    546.3   233.1
   355.7   312.5   156.    267.3   664.7   340.    236.9   298.1   445.8
   218.6   324.9   368.7  1832.5   298.4 13247.6 20997.5  9298.2  5478.8
  7822.7  9869.6 13939.3 11506.4 15194.5  9589.7 10246.3 14399.7 19987.3
 13426.4 10164.1  7965.5  8045.4  8021.5  6112.7 10614.2  8888.7 11295.1
  9611.5 10559.8 11318.8  4995.2  4985.5  6936.7  3332.3 12683.7  8814.9
  5987.7  5921.4  3099.1  5290.8  5008.3  4432.3  5710.  13874.6  6599.8
  6111.   6187.3  5836.   8668.1  7166.1  8873.2  8469.2  6864.7  8328.9
  6979.2   214.2   269.9   457.4   408.2   304.4   386.1   461.4   448.2
   605.1   239.8   429.3   450.6   466.6   452.1   330.9   302.4   607.4
  2094.3   715.3   391.4   748.5   346.8   268.

## SVR Model

In [25]:
from sklearn.svm import SVR
clf = SVR()
clf.fit(train_data_prepared, train_data_labels) 

D:\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
  gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
  tol=0.001, verbose=False)

In [27]:
some_data = temp_data.iloc[:5]
some_labels = train_data_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("예측:", clf.predict(some_data_prepared))
print(some_labels)

예측: [6281.52328698 6287.43266756 6293.26016267 6298.97446499 6304.54378194]
0     5829
1     7455
2    10086
3     5929
4    14913
Name: quantity, dtype: int64


In [ ]:
result = sgd.predict(test_data_prepared)

## KNeighborsRegressor 모델

In [34]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor()
# neigh.fit(train_data_prepared, train_data_labels) 

In [35]:
from sklearn.model_selection  import GridSearchCV
k_range = list(range(1, 31))
param_grid = dict(n_neighbors=k_range)
grid = GridSearchCV(neigh, param_grid, cv=10, scoring='neg_mean_squared_error')

In [36]:
grid.fit(train_data_prepared, train_data_labels)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [37]:
some_data = temp_data.iloc[:5]
some_labels = train_data_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("예측:", grid.predict(some_data_prepared))
print(some_labels)

예측: [ 5829.  7455. 10086.  5929. 14913.]
0     5829
1     7455
2    10086
3     5929
4    14913
Name: quantity, dtype: int64


In [38]:
grid.best_score_

-2189650.640516039

In [518]:
result = grid.predict(test_data_prepared)

# 제출 파일 생성

In [65]:
result = np.around(result)

In [66]:
result_int = result.astype(int)

In [67]:
result_int

array([ 3894, 11357,  7776, ...,    15,    13,     0])

### 서울지역 제출

In [40]:
submit = pd.read_csv('./data/submit_example.csv', encoding='utf-8', names=["date", "desti"])
result_pd = pd.DataFrame(result_int)
submit_csv = pd.concat([submit, result_pd], axis=1)
submit_csv.to_csv('submit.csv', header=False, index=False)

### 경인지역 제출

In [68]:
submit_2 = pd.read_csv('./data/submit_example2.csv', encoding='utf-8', names=["date", "desti"])
result_pd = pd.DataFrame(result_int)
submit_csv_2 = pd.concat([submit_2, result_pd], axis=1)
submit_csv_2.to_csv('submit_2.csv', header=False, index=False)

## 케라스 딥러닝 모델

In [289]:
# For deep learning model 
import keras
from keras.layers import Dense, Input, concatenate, Dropout
from keras.models import Sequential, Model
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras import metrics
from keras import backend as K

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [290]:
train_data_labels.values

array([ 5829,  7455, 10086, ..., 16106, 11191,  5884], dtype=int64)

In [291]:
X = train_data_prepared
Y = train_data_labels.values

In [294]:
num_input = Input(shape=(len(X[0]),), name='num_input')
x = Dense((1024), activation='relu')(num_input)
x = Dropout(0.3)(x)
x = Dense((512), activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense((256), activation='relu')(x)
x = Dropout(0.3)(x)
num_output = Dense(1, name='num_output')(x)

model = Model(inputs=num_input, outputs=num_output)

model.compile(optimizer='adam',
              loss='mse',
              metrics=['accuracy'])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=10, verbose=1, factor=0.5, min_lr=0.00000001)

callbacks = [
    learning_rate_reduction, # learning_rate를 점차 감소시킴으로서 최적값에 접근하는 방식
#     EarlyStopping('val_loss', patience=15)# val_loss이 최적값에서 멀어지는 순간 epoch가 남아도 학습 중지
]

history = model.fit(X, Y, epochs=100, batch_size=64, callbacks=callbacks, validation_split=0.2 )

Train on 6883 samples, validate on 1721 samples
Epoch 1/100
6883/6883 [==============================] - 5s 782us/step - loss: 24019806.9369 - acc: 1.4529e-04 - val_loss: 15990906.0802 - val_acc: 0.0000e+00
Epoch 2/100
6883/6883 [==============================] - 5s 764us/step - loss: 6298805.1723 - acc: 7.2643e-04 - val_loss: 15765093.7501 - val_acc: 5.8106e-04
Epoch 3/100
6883/6883 [==============================] - 5s 699us/step - loss: 6251347.2159 - acc: 4.3586e-04 - val_loss: 15600993.8791 - val_acc: 0.0012
Epoch 4/100
6883/6883 [==============================] - 6s 801us/step - loss: 6195917.1405 - acc: 5.8114e-04 - val_loss: 15758208.9367 - val_acc: 0.0017
Epoch 5/100
6883/6883 [==============================] - 6s 808us/step - loss: 6160372.7108 - acc: 8.7171e-04 - val_loss: 16865049.9187 - val_acc: 5.8106e-04
Epoch 6/100
6883/6883 [==============================] - 6s 821us/step - loss: 6040295.2510 - acc: 7.2643e-04 - val_loss: 15573848.6357 - val_acc: 0.0000e+00
Epoch 7/100

In [ ]:
model.predict()